In [1]:
import numpy as np
ntrain_preictal = 274
ntrain_interictal = 1647
ntest_preictal = 217
ntest_interictal = 775
nsample_interictal_train = int(np.floor(1647*ntest_preictal/ntest_interictal)*2)
sample_interictal_idx = np.random.choice(ntrain_interictal, nsample_interictal_train)
window_len_sec = 60
def convert_montage(x):
    montage_reference = [(0,2),(2,7),(7,12),(12,17),(1,6),(6,11),(11,16),(16,18),(7,8),(8,9),(9,10),(10,11),(0,3),(3,8),(8,13),(13,17),(1,5),(5,10),(10,15),(15,18)]
    if x.shape[0] != 19:
        raise ValueError
    else:
        montage = np.zeros((20, x.shape[1]))
        for idx, (i,j) in enumerate(montage_reference):
            montage[idx,:] = x[i,:] - x[j,:]
    return montage
def window_eeg(x, window_sec, stride_sec, samp_freq):
    nchannel = x.shape[0]
    npoint_original = x.shape[1]
    data_len_sec = npoint_original/samp_freq
    nwindow = np.floor((data_len_sec - window_sec)/stride_sec + 1)
    return nwindow
#     window_pt = int(samp_freq*window_sec)
#     windowed_eeg = np.zeros((nwindow,window_pt,nchannel))
#     for i in range(nwindow):
#         x_windowed = x[:,int(np.ceil(i*stride_sec*samp_freq)):int(np.ceil((i*stride_sec+window_sec)*samp_freq))]
#         #print(x_windowed.shape)
#         x_windowed = np.transpose(x_windowed)
#         windowed_eeg[i,:,:] = x_windowed
#     return windowed_eeg
def single_window_eeg(x, window_sec, samp_freq, position):
    nchannel = x.shape[0]
    npoint_original = x.shape[1]
    data_len_sec = npoint_original/samp_freq
    window_pt = int(samp_freq*window_sec)
    windowed_eeg = np.zeros((window_pt,20))
    if data_len_sec>window_sec:
        if position == 'end':
            x_windowed = x[:,npoint_original-1-window_pt:npoint_original-1]
        elif position == 'mid':
            midpoint = int(np.floor(npoint_original/2))
            halfwindow = int(np.floor(window_pt/2))
            #print(midpoint-halfwindow,midpoint+halfwindow+1)
            x_windowed = x[:,midpoint-halfwindow:midpoint+halfwindow]
        #print(x_windowed.shape)
        x_windowed = convert_montage(x_windowed)
        x_windowed = np.transpose(x_windowed)
        windowed_eeg[:,:] = x_windowed
        return True, windowed_eeg
    else:
        return False, windowed_eeg
def window_eeg_preictal(x,window_sec,stride_sec,samp_freq):
    nchannel = x.shape[0]
    npoint_original = x.shape[1]
    data_len_sec = npoint_original/samp_freq
    window_pt = int(samp_freq*window_sec)
    #windowed_eeg = np.zeros((window_pt,nchannel))
    if data_len_sec > window_sec:
        nwindow = int(np.floor((data_len_sec - window_sec)/stride_sec + 1))
        #print(nwindow, data_len_sec)
        windowed_eeg = np.zeros((nwindow,window_pt,20))
        for i in range(nwindow):
            x_windowed = x[:,int(np.ceil(i*stride_sec*samp_freq)):int(np.ceil((i*stride_sec+window_sec)*samp_freq))]
            x_windowed = convert_montage(x_windowed)
            x_windowed = np.transpose(x_windowed)
            windowed_eeg[i,:,:] = x_windowed
        return True, windowed_eeg
    else:
        return False, []    
all_train_x = []
all_train_y = []
n_valid_train_preictal = 0
n_valid_train_interictal = 0

# for i in range(ntrain_preictal):
#     train_preitctal_file_name = 'Processed_data_10s\\train\\TUH_preictal_{0}.npy'.format(str(i+1).zfill(3))
#     raw = np.load(train_preitctal_file_name)
#     valid, windowed = single_window_eeg(raw, window_len_sec, 200, 'end')
#     if valid:
#         all_train_x.append(windowed)
#         all_train_y.append(1)
#         n_valid_train_preictal += 1
            
# for i in sample_interictal_idx:
#     train_interictal_file_name = 'Processed_data_60s\\train\\TUH_interictal_{0}.npy'.format(str(i+1).zfill(3))
#     raw = np.load(train_interictal_file_name)
#     valid, windowed = single_window_eeg(raw, window_len_sec, 200, 'mid')
#     if valid:
#         all_train_x.append(windowed)
#         all_train_y.append(0)
#         n_valid_train_interictal += 1
# print('number of training preictal data: ', n_valid_train_preictal, ', number of training interictal data: ', n_valid_train_interictal)
    


for i in range(ntrain_preictal):
    train_preitctal_file_name = 'Processed_data_10s\\train\\TUH_preictal_{0}.npy'.format(str(i+1).zfill(3))
    raw = np.load(train_preitctal_file_name)
    valid, windowed = window_eeg_preictal(raw, window_len_sec, window_len_sec, 200)
    if valid:
        for j in range(max(windowed.shape[0]-5,0),windowed.shape[0]):
            all_train_x.append(windowed[j,:,:])
            all_train_y.append(1)
            n_valid_train_preictal += 1

print(n_valid_train_preictal)
            
nsample_interictal_train = int(np.floor(1647*ntest_preictal/ntest_interictal)*2)
sample_interictal_idx = np.random.choice(ntrain_interictal, nsample_interictal_train)

for i in range(ntrain_interictal):
    train_interictal_file_name = 'Processed_data_10s\\train\\TUH_interictal_{0}.npy'.format(str(i+1).zfill(3))
    raw = np.load(train_interictal_file_name)
    valid, windowed = single_window_eeg(raw, window_len_sec, 200, 'mid')
    if valid:
        all_train_x.append(windowed)
        all_train_y.append(0)
        n_valid_train_interictal += 1
print('number of training preictal data: ', n_valid_train_preictal, ', number of training interictal data: ', n_valid_train_interictal)
    
# train_interictal_file_name = 'Processed_data\train\TUH_interictal_{0}.npy'.format(str(i+1).zfill(3))
# test_preitctal_file_name = 'Processed_data\test\TUH_preictal_eval_{0}.npy'.format(str(i+1).zfill(3))
# test_interictal_file_name = 'Processed_data\test\TUH_interictal_eval_{0}.npy'.format(str(i+1).zfill(3))

563
number of training preictal data:  563 , number of training interictal data:  1558


In [9]:
np.array(all_train_x).shape

(2371, 6000, 20)

In [10]:
all_train_x = np.array(all_train_x)
import pickle
processed_data = {'X':all_train_x, 'Y':all_train_y}
pickle.dump(processed_data, open( "TUH_train_multiple_limited5_30s_window_10s_preictal_montage.p", "wb"), protocol=4)

In [2]:
all_test_x = []
all_test_y = []
n_valid_test_preictal = 0
n_valid_test_interictal = 0


for i in range(ntest_preictal):
    test_preitctal_file_name = 'Processed_data_10s\\eval\\TUH_preictal_eval_{0}.npy'.format(str(i+1).zfill(3))
    raw = np.load(test_preitctal_file_name)
    valid, windowed = single_window_eeg(raw, window_len_sec, 200, 'end')
    if valid:
        all_test_x.append(windowed)
        all_test_y.append(1)
        n_valid_test_preictal += 1

for i in range(ntest_interictal):
    test_interictal_file_name = 'Processed_data_10s\\eval\\TUH_interictal_eval_{0}.npy'.format(str(i+1).zfill(3))
    raw = np.load(test_interictal_file_name)
    valid, windowed = single_window_eeg(raw, window_len_sec, 200, 'mid')
    if valid:
        all_test_x.append(windowed)
        all_test_y.append(0)
        n_valid_test_interictal += 1
        
# for i in range(ntest_preictal):
#     test_preitctal_file_name = 'Processed_data_10s\\eval\\TUH_preictal_eval_{0}.npy'.format(str(i+1).zfill(3))
#     raw = np.load(test_preitctal_file_name)
#     valid, windowed = window_eeg_preictal(raw, window_len_sec, window_len_sec, 200)
#     if valid:
#         for j in range(windowed.shape[0]):
#             all_test_x.append(windowed[j,:,:])
#             all_test_y.append(1)
#             n_valid_test_preictal += 1

# print(n_valid_test_preictal)
            
# for i in range(ntest_interictal):
#     test_interictal_file_name = 'Processed_data_10s\\eval\\TUH_interictal_eval_{0}.npy'.format(str(i+1).zfill(3))
#     raw = np.load(test_interictal_file_name)
#     valid, windowed = single_window_eeg(raw, window_len_sec, 200, 'mid')
#     if valid:
#         all_test_x.append(windowed)
#         all_test_y.append(0)
#         n_valid_test_interictal += 1
        
all_test_x = np.array(all_test_x)
print('number of testing preictal data: ', n_valid_test_preictal, ', number of testing interictal data: ', n_valid_test_interictal)


number of testing preictal data:  169 , number of testing interictal data:  737


In [5]:
import pickle
processed_data = {'X':all_test_x, 'Y':all_test_y}
pickle.dump(processed_data, open( "TUH_test_30s_window_10s_preictal_montage.p", "wb"), protocol=4)

In [12]:
import numpy as np
import pickle
test_data = pickle.load(open("TUH_train_multiple_30s_window_10s_preictal_montage.p", "rb" ))
seed = 7
np.random.seed(seed)
X_test = test_data['X']
Y_test = test_data['Y']

In [13]:
print(X_test.shape, len(Y_test))

(3358, 6000, 20) 3358


In [16]:
X_test[300,:,:].shape

(6000, 20)

In [15]:
X_test[500,:,:]

array([[ 57.54115815,  17.86567256,   3.22090462, ...,  46.18259042,
        -13.96029674,  -4.45952443],
       [ 64.4660432 ,  16.03967081,   0.63338578, ...,  27.56865711,
          0.08083024,  -1.1880225 ],
       [ 44.64093162,  15.79788576,   1.99213829, ...,  30.46642213,
          2.55991552,  -2.25119333],
       ..., 
       [  8.97556633, -13.89729529,  -3.36985025, ...,   1.92073551,
         -0.45862479,   1.40794981],
       [  5.81334073, -11.72865106,  -3.13626291, ...,   7.66389179,
         -0.43773885,   0.94450827],
       [ -4.09532703,  -5.51489496,  -3.63884208, ...,   1.66065543,
          2.90978657,   3.30354866]])

In [3]:
a = [i for i in range(5)]
a[len(a)-1:len(a)]

[4]